In [1]:
# save working directory
address = %pwd
print(address)

/Users/Brogan/Desktop/Classifier/Random_forest


In [8]:
ls

New_data_ch.csv               Random_forest_new_data.ipynb
New_features.csv              Untitled.ipynb


In [3]:
# Libraries required

import pandas as pd
import numpy as np # Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit

# Common imports
import numpy as np
import os

# To plot lit figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

In [12]:
# Read in the features database 
file_name = address + "/New_data_ch.csv"
df = pd.read_csv(file_name)
df.head()


,Unnamed: 0,Locus,A,C,D,E,F,G,H,I,...,a,c,g,t,Gene_length,Unique_kmer_count,Charge_pH7,Gene_family,Chr_pos,gc_ratio
0,0,AT1G01010,3.030303,0.932401,6.759907,7.692308,3.962704,5.827506,2.797203,6.293706,...,29.689922,20.310078,23.798450,26.201550,1290,1036,-18.411059,Undefined,1,1.000000
1,1,AT1G01020,7.692308,1.923077,0.961538,2.884615,7.692308,2.884615,2.884615,16.346154,...,27.936508,19.365079,16.507937,36.190476,315,291,3.745583,Undefined,1,1.000000
2,2,AT1G01030,2.985075,0.597015,8.059701,4.477612,3.283582,8.059701,2.388060,3.283582,...,29.712163,23.676880,22.098422,24.512535,1077,861,-6.776819,Undefined,1,1.161765
3,3,AT1G01040,7.434555,1.832461,5.863874,8.534031,3.612565,6.701571,2.303665,4.083770,...,28.987784,17.452007,26.317627,27.242583,5730,2711,-28.618774,Undefined,1,0.892473
4,4,AT1G01050,5.188679,1.415094,7.547170,8.490566,2.830189,3.773585,2.830189,7.547170,...,28.169014,22.535211,22.065728,27.230047,639,558,-4.961909,Undefined,1,1.085366


In [13]:
# Drop the first column (index problem), and Locus position - as these cannot be fitted
df.rename(columns={ df.columns[0]: "drop" }, inplace = True)
gc_data = df.drop(["Locus","drop"], axis = 1)
gc_data.head()

,A,C,D,E,F,G,H,I,K,L,...,a,c,g,t,Gene_length,Unique_kmer_count,Charge_pH7,Gene_family,Chr_pos,gc_ratio
0,3.030303,0.932401,6.759907,7.692308,3.962704,5.827506,2.797203,6.293706,5.594406,5.827506,...,29.689922,20.310078,23.798450,26.201550,1290,1036,-18.411059,Undefined,1,1.000000
1,7.692308,1.923077,0.961538,2.884615,7.692308,2.884615,2.884615,16.346154,2.884615,12.500000,...,27.936508,19.365079,16.507937,36.190476,315,291,3.745583,Undefined,1,1.000000
2,2.985075,0.597015,8.059701,4.477612,3.283582,8.059701,2.388060,3.283582,4.179104,8.059701,...,29.712163,23.676880,22.098422,24.512535,1077,861,-6.776819,Undefined,1,1.161765
3,7.434555,1.832461,5.863874,8.534031,3.612565,6.701571,2.303665,4.083770,7.068063,8.795812,...,28.987784,17.452007,26.317627,27.242583,5730,2711,-28.618774,Undefined,1,0.892473
4,5.188679,1.415094,7.547170,8.490566,2.830189,3.773585,2.830189,7.547170,6.603774,8.962264,...,28.169014,22.535211,22.065728,27.230047,639,558,-4.961909,Undefined,1,1.085366


In [14]:
# Split data on chromosome
split = StratifiedShuffleSplit(n_splits = 1, test_size=0.2, random_state =42)
for train_index, test_index in split.split(gc_data, gc_data["Chr_pos"]):
    strat_train_set = gc_data.loc[train_index]
    strat_test_set = gc_data.loc[test_index]

In [17]:
# Check the data 
print(strat_train_set.head())

# Work out the percentage of each chromosome in each strat. It seems to have split the data correctly
print("Strat train chromosome sampling: ")
print(strat_test_set["Chr_pos"].value_counts()/len(strat_test_set), "\n")

print("Whole data chromosome frequencies: ")
print(gc_data["Chr_pos"].value_counts()/len(gc_data))

              A         C         D         E         F         G         H  \
4160   5.904762  1.904762  5.904762  5.714286  6.285714  6.095238  2.285714   
7044   8.998733  1.774398  6.210393  9.252218  3.929024  5.956907  1.394170   
20692  3.713528  1.326260  6.896552  8.222812  4.244032  5.835544  2.387268   
10288  4.843305  0.854701  3.703704  7.122507  5.128205  4.558405  2.849003   
7159   6.077348  2.762431  3.867403  3.867403  3.867403  2.209945  5.524862   

              I         K          L  ...          a          c          g  \
4160   5.904762  4.761905  11.428571  ...  27.566540  20.532319  22.686946   
7044   4.816223  7.984791   8.111534  ...  27.341772  20.042194  26.286920   
20692  4.244032  8.753316   7.957560  ...  35.537919  17.636684  22.927690   
10288  5.698006  7.692308   9.116809  ...  31.723485  20.170455  20.928030   
7159   3.314917  4.972376   8.839779  ...  30.952381  27.106227  17.948718   

               t  Gene_length  Unique_kmer_count  Charge

In [44]:
# Need to split data into predictors and labels 
gc_data = strat_train_set.drop("gc_ratio",axis =1) 
gc_labels = strat_train_set["gc_ratio"].copy()
print(len(gc_labels))

21955


In [24]:
# First need to single out the num variables before scaling 
num_gc = gc_data.drop(["Chr_pos", "Gene_family"], axis = 1)
num_gc.head()

,A,C,D,E,F,G,H,I,K,L,...,V,W,Y,a,c,g,t,Gene_length,Unique_kmer_count,Charge_pH7
4160,5.904762,1.904762,5.904762,5.714286,6.285714,6.095238,2.285714,5.904762,4.761905,11.428571,...,7.047619,0.952381,2.857143,27.566540,20.532319,22.686946,29.214195,1578,1256,-9.473728
7044,8.998733,1.774398,6.210393,9.252218,3.929024,5.956907,1.394170,4.816223,7.984791,8.111534,...,7.984791,0.380228,2.661597,27.341772,20.042194,26.286920,26.329114,2370,1603,-23.503528
20692,3.713528,1.326260,6.896552,8.222812,4.244032,5.835544,2.387268,4.244032,8.753316,7.957560,...,5.305040,1.856764,4.244032,35.537919,17.636684,22.927690,23.897707,1134,880,-3.697470
10288,4.843305,0.854701,3.703704,7.122507,5.128205,4.558405,2.849003,5.698006,7.692308,9.116809,...,7.122507,1.994302,1.709402,31.723485,20.170455,20.928030,27.178030,1056,862,12.394028
7159,6.077348,2.762431,3.867403,3.867403,3.867403,2.209945,5.524862,3.314917,4.972376,8.839779,...,2.762431,1.104972,2.762431,30.952381,27.106227,17.948718,23.992674,546,472,10.335974


In [25]:
# Pipeline for data preperation (one hot encode for "chr positon", scaler for num variables)

# Required libraries 
from sklearn.preprocessing import OneHotEncoder 
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer


num_attribs = list(num_gc)
cat_attribs = ["Chr_pos", "Gene_family"]


# Pipeline for variables which have numbers
num_pipeline = Pipeline([('std_scaler', StandardScaler()),])

full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
        ("cat", OneHotEncoder(), cat_attribs),])


gc_prepared = full_pipeline.fit_transform(gc_data)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [27]:
# Import and random forest model

from sklearn.ensemble import RandomForestRegressor
r_forest_reg = RandomForestRegressor()

r_forest_reg.fit(gc_prepared, gc_labels)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [28]:
# Predict values with random forest
y_pred = r_forest_reg.predict(gc_prepared)
y_actual = gc_labels

In [29]:
# Use RMSE to calculate the accuracy of the model 
from sklearn.metrics import mean_squared_error
forest_mse = mean_squared_error(y_actual, y_pred)
forest_rmse = np.sqrt(forest_mse)
print(forest_rmse)

0.08454228585317199


In [53]:
errors = []
zipped = zip(y_pred, y_actual)
for pred, act in zipped:
    error = abs(pred - act)
    errors.append(error)
print(len(errors))
print(sum(errors)/len(errors)*100)

21955
3.844030406261433


In [73]:
# How many times was 0 predicted successfully 

pred_0 = []
num_0 = []
pred_e = []

zipped = zip(y_pred, y_actual)
for pred, act in zipped:
    
    if act == 0.0:
        num_0.append(act)
        
        if act == pred:
            pred_0.append("correct")
        
        else:
            pred_e.append("error")
    
    else:
        continue
            

number_of_zeros = (len(num_0))
correct_zero_pred = (len(pred_0))
incorrect_zero_pred = (len(pred_e))
accuracy = (correct_zero_pred/number_of_zeros)*100

print(" Accuracy of zero prediction was  = {:.2f}%".format( accuracy ))


 Accuracy of zero prediction was  = 94.04%


In [74]:
# Check cross fold RMSE of standard Randomforest regressor 
# Lets have a look at cross fold 

from sklearn.model_selection import cross_val_score
scores = cross_val_score(r_forest_reg, gc_prepared, gc_labels,
                        scoring = "neg_mean_squared_error", cv = 10)

forest_rmse_scores = np.sqrt(-scores)

def display_scores(scores):
    print("Scores:", scores)
    print("\n")
    print("Mean:", scores.mean())
    print("\n")
    print("Standard deviation:", scores.std())

display_scores(forest_rmse_scores)

Scores: [0.19612432 0.19736362 0.20484916 0.20869662 0.1970884  0.19058123
 0.20538314 0.19676901 0.21041073 0.20847635]


Mean: 0.20157425734091955


Standard deviation: 0.0064278614308878644


In [75]:
# Use grid search to find optimum parameters for randomforest.

from sklearn.model_selection import GridSearchCV

# train across 5 folds, that's a total of (12+6)*5=90 rounds of training 
param_grid = [
    
    {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8,16,32]},
    
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4,6,8]},]

forest_reg = RandomForestRegressor(random_state=42)
grid_search = GridSearchCV(forest_reg, param_grid, cv=5,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)
grid_search.fit(gc_prepared, gc_labels)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
           oob_score=False, random_state=42, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid=[{'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8, 16, 32]}, {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4, 6, 8]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='neg_mean_squared_error', verbose=0)

In [76]:
# Results from the cross validation using a parameter grid
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

df_search = pd.DataFrame(grid_search.cv_results_)

0.33133440619898574 {'max_features': 2, 'n_estimators': 3}
0.2691947849708616 {'max_features': 2, 'n_estimators': 10}
0.24114607862296494 {'max_features': 2, 'n_estimators': 30}
0.28325884150485564 {'max_features': 4, 'n_estimators': 3}
0.2370056815194033 {'max_features': 4, 'n_estimators': 10}
0.22580859407268877 {'max_features': 4, 'n_estimators': 30}
0.2742887857839715 {'max_features': 6, 'n_estimators': 3}
0.22309545986117535 {'max_features': 6, 'n_estimators': 10}
0.21007559128576017 {'max_features': 6, 'n_estimators': 30}
0.2499189439750205 {'max_features': 8, 'n_estimators': 3}
0.21247521960894694 {'max_features': 8, 'n_estimators': 10}
0.20148978447361135 {'max_features': 8, 'n_estimators': 30}
0.23502799066345537 {'max_features': 16, 'n_estimators': 3}
0.20480851125824642 {'max_features': 16, 'n_estimators': 10}
0.19573719796648897 {'max_features': 16, 'n_estimators': 30}
0.22491180116126963 {'max_features': 32, 'n_estimators': 3}
0.20201534329974602 {'max_features': 32, 'n_es

In [77]:
# Calculate featute importance (Numbers are chromosome position)
feature_importances = grid_search.best_estimator_.feature_importances_
print(len(feature_importances))

193


In [78]:
cat_encoder = full_pipeline.named_transformers_["cat"]
cat_one_hot_attribs = list(cat_encoder.categories_[0])
attributes = num_attribs + cat_one_hot_attribs
sorted_feature_results = sorted(zip(feature_importances, attributes), reverse=True)
sorted_feature_results

[(0.27691659739661806, 1),
 (0.1948826571046037, 5),
 (0.12790773750876144, 2),
 (0.10570848143677489, 4),
 (0.03153315803527462, 3),
 (0.005699746759064864, 'T'),
 (0.005681855228004694, 'c'),
 (0.005661711838368549, 'M'),
 (0.0053146285982214195, 'S'),
 (0.005277100090722992, 'A'),
 (0.005184114594399107, 'P'),
 (0.005152672350033842, 'I'),
 (0.005139522359177675, 'Charge_pH7'),
 (0.005095531375583855, 't'),
 (0.005045724088598724, 'Y'),
 (0.005023874947449279, 'F'),
 (0.005009163366095972, 'g'),
 (0.005008293761789245, 'G'),
 (0.00498271447207196, 'Q'),
 (0.00497376966183212, 'L'),
 (0.0049556666691529016, 'R'),
 (0.004946571077501166, 'V'),
 (0.0049352951260980766, 'H'),
 (0.00492297141039355, 'Unique_kmer_count'),
 (0.004904255574274775, 'N'),
 (0.00478750168997071, 'a'),
 (0.004674652292952435, 'W'),
 (0.004669090594490087, 'K'),
 (0.004586067870538515, 'D'),
 (0.004559605042756998, 'C'),
 (0.0043151361866021224, 'E'),
 (0.0043114496339688455, 'Gene_length')]

In [79]:
# Open file to write results to
from datetime import datetime
now = datetime.now()
dt_string = now.strftime("%d-%m-%Y")
file_name = "Random_forest_results_" + dt_string
file = open(file_name, "w")
file.write("Feature,Importance\n")
for importance, feature in sorted_feature_results:
    feature = str(feature)
    importance = str(importance)
    line = feature + "," + importance + "\n"
    file.write(line)
file.close()

In [80]:
# Find the best estimator from grid search  
final_model = grid_search.best_estimator_
final_model

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features=32, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=30, n_jobs=None, oob_score=False, random_state=42,
           verbose=0, warm_start=False)

In [83]:
# Prepare the test data 

# Drop our label from the dataset to get out predictors 
X_test = strat_test_set.drop("gc_ratio", axis =1)

# Extract our label
y_test = strat_test_set["gc_ratio"].copy()

# prepare the predictors 
num_attribs = list(num_gc)
cat_attribs = ["Chr_pos", "Gene_family"]

# Pipeline for variables which have numbers
num_pipeline = Pipeline([('std_scaler', StandardScaler()),])

full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
        ("cat", OneHotEncoder(), cat_attribs),])


X_test_prep = full_pipeline.fit_transform(X_test)

# Predict the labels
y_pred = final_model.predict(X_test_prep)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


ValueError: Number of features of the model must match the input. Model n_features is 193 and input n_features is 166 

In [ ]:
# Gene family a massive predictor!
# Need to get more data on this.
# Try orthofinder on just arabidopsis asign each orthogroup consitiuent to the same number
# That number can be a representative of gene family.
